In [26]:
from chromosome import *
from itertools import permutations
import random

In [27]:
def read_distances():
    filename = "tsp_data/distances.txt"
    with open(filename) as f:
        distances = [[int(d) for d in x.split()] for x in f.readlines()]

    return distances 

def read_solution():
    filename = "tsp_data/solution.txt"
    with open(filename) as f:
        solution = [int(x) - 1 for x in f.readlines()] # subtract -1 because the index starts from 1
    return solution

In [28]:
solution = read_solution()

In [29]:
# list of list. distances[i] contains all the distances from i to every other city
distances = read_distances()

In [30]:
# optimal_cost = 0
# for city_1, city_2 in zip(solution[:-1], solution[1:]):
#     print(city_1, city_2)
#     optimal_cost += distances[city_1][city_2]

optimal_cost = sum([distances[city_1][city_2] for city_1, city_2 in zip(solution[:-1], solution[1:])])
print(optimal_cost)

33551


In [31]:
cities = [i for i in range(len(distances))]

In [32]:
class TSPIndividual(Individual):
    distances: list[list[int]] = distances

    def __init__(self, cities: list[int]):
        super().__init__(list(cities))

    def fitness(self) -> int:
        """ 
        Compute the distance encoded in the chromosome.
        This value must be minimized.
        """
        distance = 0
        for city1, city2 in zip(self.chromosome[:-1], self.chromosome[1:]):
            distance += self.distances[city1][city2]
        # add the distance from the last city back to the first one
        # print(self.chromosome[-1], self.chromosome[0])
        distance += self.distances[self.chromosome[-1]][self.chromosome[0]]
        return -distance

class TSPChromosomeInit:
    def __init__(self, cities):
        self.cities = list(cities)
        # self.g_cities = permutations(cities)

    def __call__(self) -> list[int]:
        random.shuffle(self.cities)
        return TSPIndividual(list(self.cities))
    

def swap_adjacent(x: TSPIndividual, mutation_chance: float):
    i = random.randint(0, len(x)-1)
    x[i], x[(i + 1) % len(x)] = x[(i + 1) % len(x)], x[i]

def swap_any(x: TSPIndividual, mutation_chance: float):
    i = random.randint(0, len(x)-1)
    j = random.randint(0, len(x)-1)
    x[i], x[j] = x[j], x[i]

def tsp_simple_crossover(x: TSPIndividual, y: TSPIndividual) -> TSPIndividual:
    cut_point = random.randint(1, len(x) - 2) # at least one gene is taken from each parent (that's why 1 : len - 2)
    new_chromosome = x.chromosome[0 : cut_point]
    genes_from_x = set(new_chromosome)
    for chr in y.chromosome:
        if chr not in genes_from_x:
            new_chromosome.append(chr)
    return TSPIndividual(new_chromosome)

class TSPMatingSelection:
    def __init__(self, k = 15):
        self.k = k

    def __call__(self, population: list[Individual]) -> Individual:
        return tournament_selection(population, self.k)

In [40]:
def best_fitness(individuals: list[TSPIndividual]) -> int:
    return max([x.fitness() for x in individuals])

In [36]:
tsp_init = TSPChromosomeInit(cities)
tsp_mutation = MutationOperator(swap_any, 0.5)
tsp_crossover = CrossoverOperator(tsp_simple_crossover, 0.2)
tsp_mating_selection = TSPMatingSelection(k=15)
start_population = init_population(1000, tsp_init)

In [37]:
best_fitness(start_population)

-186457

In [38]:
iterations = 1000
population = list(start_population)
for i in range(iterations):
    # print(f"Iter {i}   best fitness: {best_fitness(population)}")
    population = natural_selection(population, None, tsp_mating_selection, tsp_crossover, [tsp_mutation])

In [41]:
best_fitness(population)

-88334